In [329]:
import sys, os, re; sys.path = (["../src/", "../"] if re.match(r'^(\w\:\\)|(/)', os.getcwd()) else [])+ sys.path 

import croniter
import pandas as pd
import datetime

In [351]:
now = datetime.datetime.now()
# iter = croniter.croniter('55 23 * * Sat 55', pd.Timestamp('2023-08-25'))
iter = croniter.croniter('* * * * * 55', pd.Timestamp('2023-08-25 12:00'))

# next_datetime = croniter.get_next(datetime.datetime) # datetime not now!

# remaining_seconds = next_datetime - now
# remaining_seconds = remaining_seconds.total_seconds()
# print(f'seconds until next event: {remaining_seconds}')
pd.Timestamp(iter.get_prev(), unit='s'), pd.Timestamp(iter.get_next(), unit='s')

(Timestamp('2023-08-25 11:59:55'), Timestamp('2023-08-25 12:00:55'))

In [342]:
iter.get_prev()

1692918355.0

In [359]:
pd.Timestamp(iter.get_next(), unit='s')

Timestamp('2023-08-25 12:08:55')